In [1]:
from itertools import product
from collections import OrderedDict
from pyCBT.data.providers.mt.pairs import get_mt_pairs
from highcharts import Highchart

import ffn

np = ffn.core.np
pd = ffn.data.pd
plt = ffn.core.plt

In [2]:
# define constants
# time frame in periods/minutes (because data is sampled in minutes)
TIME_FRAMES = [21, 63, 252]
# define target price
PRICE = "CLOSE"
# define volume column
VOLUME = "VOLUME"

# load data pairs
pairs = get_mt_pairs()

In [3]:
# volatility = pd.DataFrame(index=sorted(pairs.keys()), columns=TIME_FRAMES)
# for symbol, tf in product(sorted(pairs.keys()), TIME_FRAMES):
#     sample = pairs[symbol].get(PRICE)[-tf:]
#     mean = np.mean(sample)
#     stdd = np.std(sample)

#     volatility.loc[symbol][tf] = stdd / mean * 100.0

In [4]:
inp_currency = "EUR"

def volatility_bars(inp_currency, pairs):
    chart_categories = filter(lambda x: inp_currency in x, sorted(pairs.keys()))

    chart_options = {
        'chart': {
            'type': 'columnrange',
            'inverted': True
        },

        'title': {
            'text': inp_currency
        },

        'xAxis': {
            'categories': chart_categories
        },

        'yAxis': {
            'title': {
                'text': 'Volatility'
            }
        },

        'tooltip': {
            'valueSuffix': '%'
        },

        'plotOptions': {
            'columnrange': {
                'dataLabels': {
                    'enabled': True,
                    'format': '{y}%'
                }
            }
        },

        'legend': {
            'enabled': False
        }
    }

    chart_dataset = []
    for symbol in chart_categories:
        sample = pairs[symbol].get(PRICE)[-21*1000:]
        mu = np.median(sample)
        sg = np.percentile(sample, [16, 84])

        chart_dataset += [[round((sg[0]/mu-1)*100, 2), round((sg[1]/mu-1)*100, 2)]]

    chart = Highchart()
    chart.set_dict_options(chart_options)
    chart.add_data_set(chart_dataset, "columnrange", name="Volatility")
    return chart

In [5]:
volatility_bars("EUR", pairs)

In [6]:
volatility_bars("USD", pairs)